# Modelo de programación lineal

In [1]:
import gurobipy as grb
import pandas as pd

## Entrada de datos: Instancia 4x150x1.5x1

In [2]:
datafile="4x150x1.5x1.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

## Modelo

In [3]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Academic license - for non-commercial use only - expires 2021-06-11
Using license file C:\Users\elPatan03\gurobi.lic
Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 4682350 rows, 270000 columns and 30195200 nonzeros
Model fingerprint: 0x7ca400c0
Variable types: 0 continuous, 270000 integer (270000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 15.0000000
Presolve removed 0 rows and 45300 columns (presolve time = 5s) ...
Presolve removed 4428629 rows and 258052 columns (presolve time = 10s) ...
Presolve removed 4454297 rows and 258104 columns (presolve time = 15s) ...
Presolve removed 447182

 16283 11843  139.00000  112  105  138.00000  139.50000  1.09%   115  353s
 16868 11989  139.00000  167  151  138.00000  139.50000  1.09%   115  358s
 17360 12063  139.50000   79  230  138.00000  139.50000  1.09%   116  363s
 17723 12419  139.50000   86  207  138.00000  139.50000  1.09%   118  368s
 18276 12763  139.00000  180  117  138.00000  139.50000  1.09%   120  373s
 18999 12890  139.00000   90  155  138.00000  139.50000  1.09%   122  379s
 19464 13132  139.31394   97  178  138.00000  139.50000  1.09%   124  385s
 20097 13284  139.30230   95  208  138.00000  139.50000  1.09%   128  392s
 20594 13396     cutoff   95       138.00000  139.50000  1.09%   131  405s
 20980 13562  139.14423  106  152  138.00000  139.50000  1.09%   135  412s
 21361 13765  139.00000  175   91  138.00000  139.50000  1.09%   136  419s
 21806 14108     cutoff  103       138.00000  139.50000  1.09%   139  426s
 22557 13982  139.33667   70  229  138.00000  139.50000  1.09%   142  437s
 22782 14231     cutoff  

## Entrada de datos: Instancia 4x150x1.5x2

In [4]:
datafile="4x150x1.5x2.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [5]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 4682350 rows, 270000 columns and 30195200 nonzeros
Model fingerprint: 0x906b9f11
Variable types: 0 continuous, 270000 integer (270000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 14.0000000
Presolve removed 4 rows and 45300 columns (presolve time = 5s) ...
Presolve removed 4429282 rows and 257544 columns (presolve time = 10s) ...
Presolve removed 4451862 rows and 257616 columns (presolve time = 15s) ...
Presolve removed 4481062 rows and 257672 columns (presolve time = 20s) ...
Presolve removed 4492149 rows and 257672 columns (presolve time =

 23074  6484  135.00000   47   63  134.00000  135.00000  0.75%  90.6  255s
 24668  6552     cutoff   74       134.00000  135.00000  0.75%  93.5  264s
 25210  6553  135.00000   45   79  134.00000  135.00000  0.75%  94.3  292s
 25216  6557  135.00000   46  235  134.00000  135.00000  0.75%  94.3  295s
 25223  6562  135.00000   39   36  134.00000  135.00000  0.75%  94.2  300s
 25228  6568  135.00000   22   58  134.00000  135.00000  0.75%  95.5  305s
 25286  6618  135.00000   29   98  134.00000  135.00000  0.75%  95.6  310s
 25361  6768  135.00000   31   77  134.00000  135.00000  0.75%  95.7  315s
 25485  7073  135.00000   37   79  134.00000  135.00000  0.75%  95.8  322s
 25963  7215  135.00000   26  115  134.00000  135.00000  0.75%  96.7  328s
 26395  7370  135.00000   37   73  134.00000  135.00000  0.75%  97.6  335s
 26863  7795  135.00000   70   22  134.00000  135.00000  0.75%  98.6  341s
 28238  7032     cutoff   89       134.00000  135.00000  0.75%  98.8  347s
 28548  7168     cutoff  

 108493 19203  135.00000   78  100  134.00000  135.00000  0.75%   189 1323s
 108719 19331     cutoff   86       134.00000  135.00000  0.75%   190 1333s
 109530 19460  135.00000   54   68  134.00000  135.00000  0.75%   190 1343s
 110160 19629  135.00000   84   93  134.00000  135.00000  0.75%   191 1354s
 111170 19730 infeasible   67       134.00000  135.00000  0.75%   192 1364s
 111881 19860     cutoff   67       134.00000  135.00000  0.75%   193 1374s
 112452 19968  135.00000   72   88  134.00000  135.00000  0.75%   194 1385s
 113389 20084 infeasible   73       134.00000  135.00000  0.75%   195 1395s
 113892 20324  135.00000   63   98  134.00000  135.00000  0.75%   195 1406s
 114665 20374 infeasible   68       134.00000  135.00000  0.75%   196 1418s
 115139 20453     cutoff   68       134.00000  135.00000  0.75%   197 1436s
 116387 20537  135.00000   57  139  134.00000  135.00000  0.75%   199 1451s
 117277 20746 infeasible   69       134.00000  135.00000  0.75%   199 1469s
 118595 2074

 199241 29123     cutoff   60       134.00000  135.00000  0.75%   254 2916s
 200333 29328     cutoff   73       134.00000  135.00000  0.75%   255 2932s
 201622 29452  135.00000   65  118  134.00000  135.00000  0.75%   255 2948s
 202538 29616     cutoff   69       134.00000  135.00000  0.75%   256 2963s
 203469 29705  135.00000   73  124  134.00000  135.00000  0.75%   256 2984s
 203822 29745  135.00000   79  125  134.00000  135.00000  0.75%   256 2999s
 204615 29898     cutoff   82       134.00000  135.00000  0.75%   257 3017s
 205877 30104  135.00000   77   65  134.00000  135.00000  0.75%   257 3031s
 207033 30240     cutoff   82       134.00000  135.00000  0.75%   258 3052s
 207691 30339  135.00000   77  121  134.00000  135.00000  0.75%   258 3068s
 208905 30394  135.00000   70  140  134.00000  135.00000  0.75%   258 3085s
 209889 30555     cutoff   82       134.00000  135.00000  0.75%   258 3100s
 210866 30566  135.00000   81  122  134.00000  135.00000  0.75%   259 3115s
 211015 3062

## Entrada de datos: Instancia 4x150x1.5x3

In [6]:
datafile="4x150x1.5x3.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [7]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 4682350 rows, 270000 columns and 30195200 nonzeros
Model fingerprint: 0x01c405e6
Variable types: 0 continuous, 270000 integer (270000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 15.0000000
Presolve removed 0 rows and 45300 columns (presolve time = 5s) ...
Presolve removed 4414686 rows and 257880 columns (presolve time = 10s) ...
Presolve removed 4437070 rows and 257944 columns (presolve time = 15s) ...
Presolve removed 4465283 rows and 257976 columns (presolve time = 20s) ...
Presolve removed 4477412 rows and 257976 columns (presolve time =

## Entrada de datos: Instancia 4x150x1.5x4

In [8]:
datafile="4x150x1.5x4.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [9]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 4682350 rows, 270000 columns and 30195200 nonzeros
Model fingerprint: 0xa4615fa6
Variable types: 0 continuous, 270000 integer (270000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 14.0000000
Presolve removed 8 rows and 45300 columns (presolve time = 6s) ...
Presolve removed 4380504 rows and 256744 columns (presolve time = 10s) ...
Presolve removed 4405268 rows and 256792 columns (presolve time = 15s) ...
Presolve removed 4430841 rows and 256860 columns (presolve time = 20s) ...
Presolve removed 4445816 rows and 256860 columns (presolve time =

 27012 21297  139.33333   28  221  138.00000  139.33333  0.97%   161  537s
 27018 21300  139.33333   29  209  138.00000  139.33333  0.97%   161  543s
 27024 21302  139.00000   29  161  138.00000  139.33333  0.97%   161  546s
 27030 21304  139.33333   30  293  138.00000  139.33333  0.97%   161  553s
 27036 21311  139.00000   30  122  138.00000  139.33333  0.97%   161  560s
 27055 21323  139.00000   31  179  138.00000  139.33333  0.97%   162  565s
 27065 21330  139.33333   32  290  138.00000  139.33333  0.97%   162  570s
 27088 21343  139.33333   33  287  138.00000  139.33333  0.97%   162  577s
 27102 21348  139.00000   33  144  138.00000  139.33333  0.97%   162  581s
 27134 21369  139.00000   34  177  138.00000  139.33333  0.97%   162  587s
 27153 21379  139.33333   35  213  138.00000  139.33333  0.97%   162  590s
 27219 21427  139.33333   37  178  138.00000  139.33333  0.97%   162  595s
 27277 21461  139.00000   39  136  138.00000  139.33333  0.97%   162  601s
 27344 21499  139.33333  

## Entrada de datos: Instancia 4x150x1.5x5

In [10]:
datafile="4x150x1.5x5.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [11]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 4682350 rows, 270000 columns and 30195200 nonzeros
Model fingerprint: 0x0e986c59
Variable types: 0 continuous, 270000 integer (270000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 12.0000000
Presolve removed 0 rows and 45300 columns (presolve time = 6s) ...
Presolve removed 4402451 rows and 256972 columns (presolve time = 10s) ...
Presolve removed 4428843 rows and 257044 columns (presolve time = 15s) ...
Presolve removed 4448087 rows and 257100 columns (presolve time = 20s) ...
Presolve removed 4456289 rows and 257100 columns (presolve time =

## Entrada de datos: Instancia 4x150x2x1

In [12]:
datafile="4x150x2x1.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [13]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 4682350 rows, 270000 columns and 30195200 nonzeros
Model fingerprint: 0xec489ab1
Variable types: 0 continuous, 270000 integer (270000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 13.0000000
Presolve removed 0 rows and 45300 columns (presolve time = 6s) ...
Presolve removed 4476454 rows and 260212 columns (presolve time = 10s) ...
Presolve removed 4498750 rows and 260268 columns (presolve time = 15s) ...
Presolve removed 4518760 rows and 260324 columns (presolve time = 20s) ...
Presolve removed 4529888 rows and 260324 columns (presolve time =

 12871 11052  124.00000   22  314  121.00000  124.00000  2.48%   393  732s
 12877 11056  124.00000   22  270  121.00000  124.00000  2.48%   393  738s
 12883 11060  124.00000   23  364  121.00000  124.00000  2.48%   394  744s
 12889 11064  124.00000   23  334  121.00000  124.00000  2.48%   394  749s
 12895 11070  124.00000   24  305  121.00000  124.00000  2.48%   394  752s
 12903 11075  124.00000   24  333  121.00000  124.00000  2.48%   394  757s
 12910 11078  124.00000   25  307  121.00000  124.00000  2.48%   395  771s
 12916 11107  124.00000   25  348  121.00000  124.00000  2.48%   395  781s
 12947 11200  124.00000   27  298  121.00000  124.00000  2.48%   395  796s
 13050 11356  124.00000   33  323  121.00000  124.00000  2.48%   396  820s
 13240 11757  124.00000   44  270  121.00000  124.00000  2.48%   406  853s
 13705 12179  123.99996   71  284  121.00000  124.00000  2.48%   416  883s
 14282 12797  123.99878  107  280  121.00000  124.00000  2.48%   422  907s
 15092 13521  123.91999  

 40072 27631  122.91631  473  670  121.00000  122.91631  1.58%   483 2370s
 40074 27632  122.91231  608  657  121.00000  122.91231  1.58%   483 2377s
 40075 27633  122.50481  830  653  121.00000  122.90836  1.58%   483 2381s
 40077 27634  122.90492  199  656  121.00000  122.90492  1.57%   483 2390s
 40079 27635  122.56982  824  659  121.00000  122.90139  1.57%   483 2398s
 40080 27636  122.73431  876  671  121.00000  122.90073  1.57%   483 2401s
 40081 27637  122.90034  160  667  121.00000  122.90034  1.57%   483 2405s
 40083 27638  122.89506  323  637  121.00000  122.89506  1.57%   483 2412s
 40084 27639  122.50000  220  686  121.00000  122.89397  1.57%   483 2415s
 40086 27640  122.64195  592  676  121.00000  122.89167  1.56%   483 2421s
 40087 27641  122.88999  692  685  121.00000  122.88999  1.56%   483 2425s
 40089 27642  122.88544  503  668  121.00000  122.88544  1.56%   483 2432s
 40091 27643  122.88097  450  661  121.00000  122.88097  1.55%   483 2438s
 40092 27644  122.00000  

 40232 27739  122.45100  837  524  121.00000  122.74462  1.44%   494 2936s
 40234 27740  122.74409  448  540  121.00000  122.74409  1.44%   494 2942s
 40236 27742  122.74354  403  523  121.00000  122.74354  1.44%   494 2948s
 40237 27742  122.74262  695  598  121.00000  122.74262  1.44%   494 2952s
 40238 27743  122.20245  793  556  121.00000  122.74221  1.44%   494 2956s
 40240 27744  122.74134  178  596  121.00000  122.74134  1.44%   494 2962s
 40241 27745  122.34179  805  636  121.00000  122.74110  1.44%   494 2965s
 40243 27746  122.73887  238  643  121.00000  122.73887  1.44%   494 2972s
 40244 27747  122.00000  240  531  121.00000  122.73790  1.44%   494 2976s
 40246 27748  122.73711  299  590  121.00000  122.73711  1.44%   494 2983s
 40247 27749  122.73699   44  627  121.00000  122.73699  1.44%   494 2986s
 40248 27750  122.73661  337  638  121.00000  122.73661  1.44%   494 2992s
 40249 27750  122.73634  634  638  121.00000  122.73634  1.43%   494 2995s
 40250 27751  122.73611  

 40392 27847  122.00000  126  231  121.00000  122.70370  1.41%   504 3497s
 40393 27848  122.68488  854  248  121.00000  122.70370  1.41%   504 3501s
 40395 27849  122.70370  243  486  121.00000  122.70370  1.41%   504 3511s
 40396 27850  122.70370  692  530  121.00000  122.70370  1.41%   504 3518s
 40397 27851  122.70370  691  483  121.00000  122.70370  1.41%   504 3526s
 40398 27851  122.70370  197  460  121.00000  122.70370  1.41%   504 3532s
 40399 27852  122.70370  323  446  121.00000  122.70370  1.41%   504 3536s
 40400 27853  122.70370  493  492  121.00000  122.70370  1.41%   504 3542s
 40401 27853  122.70370  636  501  121.00000  122.70370  1.41%   504 3546s
 40402 27854  122.70370  207  503  121.00000  122.70370  1.41%   504 3550s
 40404 27855  122.70370  630  608  121.00000  122.70370  1.41%   504 3558s
 40405 27856  122.70176  143  614  121.00000  122.70176  1.41%   504 3563s
 40406 27857  122.35504  887  647  121.00000  122.70105  1.41%   504 3566s
 40407 27857  122.69981  

## Entrada de datos: Instancia 4x150x2x2

In [14]:
datafile="4x150x2x2.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [15]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Academic license - for non-commercial use only - expires 2021-06-11
Using license file C:\Users\elPatan03\gurobi.lic
Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 4682350 rows, 270000 columns and 30195200 nonzeros
Model fingerprint: 0x7fc4eeb1
Variable types: 0 continuous, 270000 integer (270000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 14.0000000
Presolve removed 0 rows and 45300 columns (presolve time = 7s) ...
Presolve removed 4481402 rows and 260600 columns (presolve time = 12s) ...
Presolve removed 4481402 rows and 260608 columns (presolve time = 15s) ...
Presolve removed 450922

     0     0  119.27859    0  633  116.00000  119.27859  2.83%     -  272s
     0     0  119.27541    0  626  116.00000  119.27541  2.82%     -  273s
     0     0  119.27466    0  633  116.00000  119.27466  2.82%     -  273s
     0     0  119.27418    0  653  116.00000  119.27418  2.82%     -  274s
     0     0  119.27399    0  674  116.00000  119.27399  2.82%     -  274s
     0     0  119.24340    0  594  116.00000  119.24340  2.80%     -  277s
     0     0  119.23766    0  618  116.00000  119.23766  2.79%     -  278s
     0     0  119.23473    0  623  116.00000  119.23473  2.79%     -  279s
     0     0  119.23373    0  660  116.00000  119.23373  2.79%     -  280s
     0     0  119.23317    0  648  116.00000  119.23317  2.79%     -  280s
     0     0  119.23297    0  634  116.00000  119.23297  2.79%     -  281s
     0     0  119.22258    0  619  116.00000  119.22258  2.78%     -  283s
     0     0  119.21553    0  657  116.00000  119.21553  2.77%     -  284s
     0     0  119.21324  

  1405  1516  118.16461  212  442  116.00000  118.95652  2.55%   759  533s
  1516  1660  118.34360  220  462  116.00000  118.95652  2.55%   733  537s
  1660  1795  118.31351  231  444  116.00000  118.95652  2.55%   707  542s
  1795  2019  118.27329  246  433  116.00000  118.95652  2.55%   678  547s
  2019  2173  118.20492  280  428  116.00000  118.95652  2.55%   631  553s
  2173  2508  118.17410  302  413  116.00000  118.95652  2.55%   605  558s
  2508  2771  118.04767  354  373  116.00000  118.95652  2.55%   552  563s
  2775  3172  117.56598  389  353  116.00000  118.95652  2.55%   520  568s
  3178  3602  117.48580  470  322  116.00000  118.95652  2.55%   474  573s
  3613  4049  117.25797  554  284  116.00000  118.95652  2.55%   429  577s
  4105  4279  117.11902  660  241  116.00000  118.95597  2.55%   392  583s
  4375  4517  118.87042    5  742  116.00000  118.95597  2.55%   381  589s
  4639  4555  118.80106    7  666  116.00000  118.95597  2.55%   371  596s
  4694  4568  118.70962  

  5169  3001  118.31988   50  677  117.00000  118.31988  1.13%   446 1115s
  5176  3005  118.31829   89  675  117.00000  118.31829  1.13%   445 1120s
  5182  3009  118.09259  375  689  117.00000  118.31722  1.13%   444 1125s
  5188  3013  118.01244  365  665  117.00000  118.31593  1.12%   444 1130s
  5194  3017  118.07369  292  681  117.00000  118.31516  1.12%   443 1135s
  5201  3023  118.08135  410  328  117.00000  118.31508  1.12%   464 1140s
  5205  3026  118.31508  205  495  117.00000  118.31508  1.12%   463 1146s
  5208  3028  118.18018  281  516  117.00000  118.31508  1.12%   463 1150s
  5211  3030  118.31508   15  537  117.00000  118.31508  1.12%   463 1155s
  5217  3034  118.31214   15  656  117.00000  118.31214  1.12%   462 1160s
  5222  3037  118.30985   26  684  117.00000  118.30985  1.12%   462 1165s
  5229  3042  118.21537  266  655  117.00000  118.30752  1.12%   461 1170s
  5234  3045  118.17979  286  638  117.00000  118.30542  1.12%   461 1175s
  5240  3049  118.12900  

 12808  6187  118.11818  180  676  117.00000  118.20155  1.03%   571 2085s
 12814  6191  118.19376  120  634  117.00000  118.20094  1.03%   571 2090s
 12815  6192  118.06248  282  613  117.00000  118.20094  1.03%   571 2096s
 12817  6193  118.16251  155  601  117.00000  118.20092  1.03%   571 2104s
 12818  6197  118.20028   89  600  117.00000  118.20028  1.03%   587 2122s
 12820  6198  118.20017   90  654  117.00000  118.20017  1.03%   587 2140s
 12824  6200  118.20016   91  659  117.00000  118.20016  1.03%   588 2162s
 12830  6202  118.19988   92  689  117.00000  118.19988  1.03%   589 2180s
 12836  6203  118.19977   93  665  117.00000  118.19977  1.03%   590 2193s
 12842  6207  118.05045   93  635  117.00000  118.19977  1.03%   593 2203s
 12848  6211  118.19955   94  699  117.00000  118.19955  1.03%   593 2210s
 12854  6213  118.03445   94  569  117.00000  118.19955  1.03%   594 2216s
 12863  6215  118.19944   95  663  117.00000  118.19955  1.03%   595 2220s
 12873  6210  118.01849  

## Entrada de datos: Instancia 4x150x2x3

In [16]:
datafile="4x150x2x3.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [17]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 4682350 rows, 270000 columns and 30195200 nonzeros
Model fingerprint: 0x32e08148
Variable types: 0 continuous, 270000 integer (270000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 15.0000000
Presolve removed 0 rows and 45300 columns (presolve time = 5s) ...
Presolve removed 4440159 rows and 259312 columns (presolve time = 10s) ...
Presolve removed 4462795 rows and 259356 columns (presolve time = 15s) ...
Presolve removed 4492510 rows and 259416 columns (presolve time = 20s) ...
Presolve removed 4504702 rows and 259416 columns (presolve time =

  6960  5117  124.00000  165  219  122.00000  125.00000  2.46%   358  481s
  7026  5291  124.00000  166  213  122.00000  125.00000  2.46%   359  486s
  7264  5400  124.00000  245  260  122.00000  125.00000  2.46%   360  492s
  7383  5605  124.00000  275  280  122.00000  125.00000  2.46%   359  498s
  7602  5858  124.00000  343  246  122.00000  125.00000  2.46%   356  505s
  7895  6033  124.00000  409  315  122.00000  125.00000  2.46%   355  511s
  8096  6245  124.00000  455  302  122.00000  125.00000  2.46%   356  518s
  8350  6300  124.00000  516  321  122.00000  125.00000  2.46%   358  529s
  8407  6455  124.00000  527  440  122.00000  125.00000  2.46%   362  536s
  8583  6785  124.00000  542  332  122.00000  125.00000  2.46%   361  543s
  8934  7003  124.00000  603  319  122.00000  125.00000  2.46%   359  551s
  9188  7564  124.00000  646  324  122.00000  125.00000  2.46%   362  560s
  9766  7724  124.00000  704  317  122.00000  125.00000  2.46%   351  570s
  9971  7919  124.00000  

## Entrada de datos: Instancia 4x150x2x4

In [18]:
datafile="4x150x2x4.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [19]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 4682350 rows, 270000 columns and 30195200 nonzeros
Model fingerprint: 0xf882c9eb
Variable types: 0 continuous, 270000 integer (270000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 16.0000000
Presolve removed 8 rows and 45300 columns (presolve time = 5s) ...
Presolve removed 4457005 rows and 260088 columns (presolve time = 10s) ...
Presolve removed 4482277 rows and 260136 columns (presolve time = 15s) ...
Presolve removed 4500292 rows and 260192 columns (presolve time = 20s) ...
Presolve removed 4509235 rows and 260192 columns (presolve time =

   647   553  122.50000   19  184  121.00000  122.50000  1.24%  1117  340s
   651   556  122.50000   12  156  121.00000  122.50000  1.24%  1111  345s
   655   558  122.24642   64  222  121.00000  122.50000  1.24%  1104  352s
   658   560  122.50000   38  293  121.00000  122.50000  1.24%  1099  355s
   662   564  122.50000   28  181  121.00000  122.50000  1.24%  1323  363s
   664   564  122.50000   29  219  121.00000  122.50000  1.24%  1321  365s
   669   565  122.50000   31  253  121.00000  122.50000  1.24%  1314  370s
   678   566  122.50000   33  232  121.00000  122.50000  1.24%  1300  375s
   716   576  122.50000   36  236  121.00000  122.50000  1.24%  1242  380s
   725   577  122.01836   36  182  121.00000  122.50000  1.24%  1231  391s
   758   591  122.37648   38  253  121.00000  122.50000  1.24%  1187  396s
   795   615  122.50000   40  219  121.00000  122.50000  1.24%  1144  400s
   954   774  122.50000   49  221  121.00000  122.50000  1.24%  1002  405s
  1165   919  122.50000  

 38575 22331  122.50000   54  263  121.00000  122.50000  1.24%   495 2044s
 38579 22336  122.50000   55  260  121.00000  122.50000  1.24%   495 2052s
 38585 22340  122.50000   56  311  121.00000  122.50000  1.24%   495 2056s
 38597 22348  122.50000   57  318  121.00000  122.50000  1.24%   495 2063s
 38603 22351  122.10391   57  264  121.00000  122.50000  1.24%   495 2066s
 38615 22359  122.30000   58  285  121.00000  122.50000  1.24%   495 2070s
 38628 22384  122.50000   59  294  121.00000  122.50000  1.24%   495 2081s
 38651 22419  122.50000   61  315  121.00000  122.50000  1.24%   495 2090s
 38694 22541  122.50000   64  366  121.00000  122.50000  1.24%   495 2112s
 38833 22938  122.50000   76  334  121.00000  122.50000  1.24%   496 2155s
 39291 24058  122.50000  102  307  121.00000  122.50000  1.24%   496 2193s
 40777 24209  122.44463  185  310  121.00000  122.50000  1.24%   490 2208s
 41678 23943  122.01374  207  333  121.00000  122.50000  1.24%   484 2228s
 41760 24117  122.30222  

## Entrada de datos: Instancia 4x150x2x5

In [20]:
datafile="4x150x2x5.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [21]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 4682350 rows, 270000 columns and 30195200 nonzeros
Model fingerprint: 0x10288025
Variable types: 0 continuous, 270000 integer (270000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 15.0000000
Presolve removed 0 rows and 45300 columns (presolve time = 5s) ...
Presolve removed 4477995 rows and 260280 columns (presolve time = 10s) ...
Presolve removed 4502063 rows and 260384 columns (presolve time = 15s) ...
Presolve removed 4527224 rows and 260384 columns (presolve time = 20s) ...
Presolve removed 4539500 rows and 260384 columns (presolve time =

## Entrada de datos: Instancia 4x150x2.5x1

In [22]:
datafile="4x150x2.5x1.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [23]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 4682350 rows, 270000 columns and 30195200 nonzeros
Model fingerprint: 0xd1d153ab
Variable types: 0 continuous, 270000 integer (270000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 14.0000000
Presolve removed 0 rows and 45300 columns (presolve time = 5s) ...
Presolve removed 4539992 rows and 262720 columns (presolve time = 10s) ...
Presolve removed 4562434 rows and 262824 columns (presolve time = 15s) ...
Presolve removed 4586253 rows and 262824 columns (presolve time = 20s) ...
Presolve removed 4597839 rows and 262824 columns (presolve time =

 11388  8551  106.46308   28  373  104.00000  106.50000  2.40%   353  382s
 11763  8657  106.00000   37  283  104.00000  106.50000  2.40%   354  421s
 11921  9026  106.00000   43  240  104.00000  106.50000  2.40%   353  427s
 12331  9107  105.50000   83  171  104.00000  106.50000  2.40%   353  434s
 12470  9253     cutoff  142       104.00000  106.50000  2.40%   353  440s
 12654  9267  105.50000   33  239  104.00000  106.50000  2.40%   362  467s
 12674 11378  105.50000   34  269  104.00000  106.50000  2.40%   364  547s
 15074 11638  105.00000  118  160  104.00000  106.50000  2.40%   420  562s
 15401 12304  105.54910  154  149  104.00000  106.50000  2.40%   421  581s
 16351 12715  106.50000   33  380  104.00000  106.50000  2.40%   428  598s
 17027 13395  105.50000   72  256  104.00000  106.50000  2.40%   429  616s
 17979 13668  105.15468   42  213  104.00000  106.50000  2.40%   432  631s
 18498 14516     cutoff   39       104.00000  106.50000  2.40%   434  647s
 19431 14899  106.14286  

 38019 26940  105.16667   56  202  104.00000  106.21429  2.13%   431 1476s
 38962 27116  105.00000  175  235  104.00000  106.21429  2.13%   427 1486s
 39588 27448  106.00000   73  222  104.00000  106.20833  2.12%   425 1496s
 40273 27388  105.00000  248   97  104.00000  106.20833  2.12%   423 1512s
 40515 27797     cutoff  289       104.00000  106.20833  2.12%   423 1523s
 41138 28184  106.00000   89  291  104.00000  106.20833  2.12%   422 1541s
 41785 28662  105.50000  142  169  104.00000  106.20833  2.12%   420 1552s
 42602 28997  105.50000  248  177  104.00000  106.20833  2.12%   417 1564s
 43389 29351  105.64286   75  246  104.00000  106.20833  2.12%   414 1575s
 44243 29129  105.50000  116  302  104.00000  106.20833  2.12%   410 1586s
 44306 29545  105.42537  125  353  104.00000  106.20833  2.12%   410 1596s
 44757 30071  105.12003  188  247  104.00000  106.20833  2.12%   410 1608s
 45478 30861  105.00000   86  175  104.00000  106.20833  2.12%   408 1621s
 46635 31009  105.00000  

 112163 75682  105.50000  121  154  104.00000  106.16667  2.08%   363 2949s
 113159 76452  105.33333  274  206  104.00000  106.16667  2.08%   363 2967s
 113998 76915  105.29345   87  195  104.00000  106.16667  2.08%   363 2986s
 114660 77291  106.00000   57  395  104.00000  106.16667  2.08%   364 3004s
 115160 77711  106.00000   95  245  104.00000  106.16667  2.08%   365 3021s
 115620 78475  105.50017  217  248  104.00000  106.16667  2.08%   365 3041s
 116431 78919  106.00000   58  378  104.00000  106.16667  2.08%   366 3059s
 116935 79310  105.89037   74  420  104.00000  106.16667  2.08%   366 3080s
 117403 79989  105.68283  105  404  104.00000  106.16667  2.08%   367 3101s
 118114 80421  105.46134  161  348  104.00000  106.16667  2.08%   368 3123s
 118624 81169  105.29403  239  413  104.00000  106.16667  2.08%   369 3146s
 119493 81509  105.22129  292  327  104.00000  106.16667  2.08%   369 3168s
 119927 81791  105.00000  366  207  104.00000  106.16667  2.08%   370 3191s
 120257 8230

## Entrada de datos: Instancia 4x150x2.5x2

In [24]:
datafile="4x150x2.5x2.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [25]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 4682350 rows, 270000 columns and 30195200 nonzeros
Model fingerprint: 0x2dbbae55
Variable types: 0 continuous, 270000 integer (270000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 11.0000000
Presolve removed 0 rows and 45300 columns (presolve time = 5s) ...
Presolve removed 4490902 rows and 261552 columns (presolve time = 10s) ...
Presolve removed 4514814 rows and 261652 columns (presolve time = 15s) ...
Presolve removed 4537817 rows and 261652 columns (presolve time = 20s) ...
Presolve removed 4553570 rows and 261652 columns (presolve time =

     0     0  105.25000    0   19  104.00000  105.25000  1.20%     -   92s
     0     0  105.25000    0  101  104.00000  105.25000  1.20%     -   93s
     0     0  105.25000    0  196  104.00000  105.25000  1.20%     -   93s
     0     0  105.25000    0  180  104.00000  105.25000  1.20%     -   94s
     0     0  105.25000    0  153  104.00000  105.25000  1.20%     -   94s
     0     0  105.25000    0  165  104.00000  105.25000  1.20%     -   94s
     0     0  105.25000    0  131  104.00000  105.25000  1.20%     -   94s
     0     0  105.25000    0  153  104.00000  105.25000  1.20%     -   94s
     0     0  105.25000    0   98  104.00000  105.25000  1.20%     -   94s
     0     0  105.25000    0   16  104.00000  105.25000  1.20%     -   94s
     0     0  105.25000    0   67  104.00000  105.25000  1.20%     -   95s
     0     0  105.25000    0   16  104.00000  105.25000  1.20%     -   95s
     0     0  105.25000    0   16  104.00000  105.25000  1.20%     -   95s
     0     2  105.25000  

## Entrada de datos: Instancia 4x150x2.5x3

In [26]:
datafile="4x150x2.5x3.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [27]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 4682350 rows, 270000 columns and 30195200 nonzeros
Model fingerprint: 0x0b2aa02f
Variable types: 0 continuous, 270000 integer (270000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 14.0000000
Presolve removed 0 rows and 45300 columns (presolve time = 5s) ...
Presolve removed 4488783 rows and 261284 columns (presolve time = 10s) ...
Presolve removed 4510435 rows and 261352 columns (presolve time = 15s) ...
Presolve removed 4529548 rows and 261352 columns (presolve time = 20s) ...
Presolve removed 4540146 rows and 261352 columns (presolve time =

     0     0  110.42467    0  683  107.00000  110.42467  3.20%     -  215s
     0     0  110.42443    0  687  107.00000  110.42443  3.20%     -  215s
     0     0  110.31923    0  573  107.00000  110.31923  3.10%     -  219s
     0     0  110.30313    0  561  107.00000  110.30313  3.09%     -  221s
     0     0  110.29667    0  570  107.00000  110.29667  3.08%     -  221s
     0     0  110.29551    0  601  107.00000  110.29551  3.08%     -  222s
     0     0  110.29532    0  600  107.00000  110.29532  3.08%     -  222s
     0     0  110.27391    0  596  107.00000  110.27391  3.06%     -  225s
     0     0  110.26359    0  589  107.00000  110.26359  3.05%     -  228s
     0     0  110.26061    0  558  107.00000  110.26061  3.05%     -  228s
     0     0  110.25998    0  578  107.00000  110.25998  3.05%     -  229s
     0     0  110.25967    0  600  107.00000  110.25967  3.05%     -  229s
     0     0  110.22958    0  595  107.00000  110.22958  3.02%     -  231s
     0     0  110.22659  

   475   507  109.79168   60  528  107.00000  109.99174  2.80%  2179  611s
   508   540  109.72507   65  467  107.00000  109.99174  2.80%  2134  622s
   541   564  109.74956   68  492  107.00000  109.99174  2.80%  2115  635s
   565   601  109.74837   69  468  107.00000  109.99174  2.80%  2109  647s
   602   650  109.72698   73  488  107.00000  109.99174  2.80%  2067  662s
   651   705  109.71480   83  519  107.00000  109.99174  2.80%  2013  678s
   706   772  109.70410   91  506  107.00000  109.99174  2.80%  1947  691s
   773   835  109.69219  109  465  107.00000  109.99174  2.80%  1847  705s
   836   897  109.66910  118  439  107.00000  109.99174  2.80%  1802  719s
   898   985  109.63046  128  451  107.00000  109.99174  2.80%  1757  734s
   986  1056  109.58867  146  464  107.00000  109.99174  2.80%  1678  750s
  1057  1173  109.54845  150  480  107.00000  109.99174  2.80%  1632  765s
  1174  1299  109.52559  160  432  107.00000  109.99174  2.80%  1544  781s
  1300  1449  109.50611  

  4138  3949  109.32921   56  679  107.00000  109.32921  2.18%   750 1410s
  4141  3951  108.81931  402  707  107.00000  109.32727  2.18%   749 1415s
  4145  3954  109.32435   18  656  107.00000  109.32435  2.17%   749 1420s
  4149  3957  108.11856  637  716  107.00000  109.32318  2.17%   748 1425s
  4155  3962  109.32283  192  318  107.00000  109.32283  2.17%   789 1431s
  4157  3964  109.32283    9  471  107.00000  109.32283  2.17%   789 1435s
  4160  3966  109.09068  210  481  107.00000  109.32283  2.17%   788 1442s
  4162  3967  109.16693  288  531  107.00000  109.32283  2.17%   788 1445s
  4165  3969  108.01529  707  545  107.00000  109.32283  2.17%   787 1450s
  4169  3972  108.55609  464  559  107.00000  109.32283  2.17%   787 1457s
  4171  3973  108.71460  342  593  107.00000  109.32283  2.17%   786 1460s
  4174  3975  109.32283  165  608  107.00000  109.32283  2.17%   786 1466s
  4177  3977  109.32283  107  679  107.00000  109.32283  2.17%   785 1470s
  4180  3979  109.31463  

 12888  9470  109.14299  115  597  107.00000  109.28244  2.13%   695 2570s
 12891  9472  108.40237  829  595  107.00000  109.28244  2.13%   695 2576s
 12893  9474  108.49450  442  628  107.00000  109.28244  2.13%   695 2581s
 12895  9475  109.23173  106  665  107.00000  109.28244  2.13%   695 2587s
 12897  9476  109.14303  134  655  107.00000  109.28244  2.13%   695 2590s
 12900  9478  109.09873  224  673  107.00000  109.28244  2.13%   695 2595s
 12903  9480  109.14810   88  681  107.00000  109.28244  2.13%   694 2600s
 12907  9483  108.45052   62  702  107.00000  109.27799  2.13%   694 2606s
 12909  9484  108.19818 1043  616  107.00000  109.27427  2.13%   694 2610s
 12912  9486  109.02190  204  675  107.00000  109.26943  2.12%   694 2615s
 12916  9489  108.28634  629  701  107.00000  109.26559  2.12%   694 2620s
 12920  9492  109.20867   80  750  107.00000  109.26169  2.11%   693 2626s
 12923  9494  108.75002  799  692  107.00000  109.25939  2.11%   693 2631s
 12926  9496  108.30265  

  StrongCG: 99
  Flow cover: 204
  GUB cover: 9
  Zero half: 13
  RLT: 2
  Relax-and-lift: 10

Explored 14016 nodes (11191975 simplex iterations) in 3400.99 seconds
Thread count was 6 (of 6 available processors)

Solution count 9: 108 107 106 ... -0

Optimal solution found (tolerance 1.00e-04)
Best objective 1.080000000000e+02, best bound 1.080000000000e+02, gap 0.0000%


## Entrada de datos: Instancia 4x150x2.5x4

In [28]:
datafile="4x150x2.5x4.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [29]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 4682350 rows, 270000 columns and 30195200 nonzeros
Model fingerprint: 0x76e4165e
Variable types: 0 continuous, 270000 integer (270000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 14.0000000
Presolve removed 0 rows and 45300 columns (presolve time = 5s) ...
Presolve removed 4509087 rows and 261764 columns (presolve time = 10s) ...
Presolve removed 4531275 rows and 261860 columns (presolve time = 15s) ...
Presolve removed 4553371 rows and 261860 columns (presolve time = 20s) ...
Presolve removed 4563348 rows and 261860 columns (presolve time =

     0     0  110.60049    0  458  107.00000  110.60049  3.36%     -  157s
     0     0  110.59955    0  412  107.00000  110.59955  3.36%     -  157s
     0     0  110.59932    0  399  107.00000  110.59932  3.36%     -  158s
     0     0  110.59609    0  424  107.00000  110.59609  3.36%     -  159s
     0     0  110.59573    0  424  107.00000  110.59573  3.36%     -  160s
     0     0  110.59301    0  454  107.00000  110.59301  3.36%     -  161s
     0     0  110.59301    0  408  107.00000  110.59301  3.36%     -  162s
     0     2  110.59301    0  408  107.00000  110.59301  3.36%     -  170s
    31    38  110.41035    7  401  107.00000  110.58731  3.35%   932  178s
H   32    38                     108.0000000  110.58731  2.40%   902  178s
    52    60  110.40699    9  350  108.00000  110.58731  2.40%   789  180s
    74    84  110.38040   13  332  108.00000  110.58731  2.40%   706  185s
   141   148  110.27159   21  277  108.00000  110.58731  2.40%   530  194s
   147   161  110.26068  

## Entrada de datos: Instancia 4x150x2.5x5

In [30]:
datafile="4x150x2.5x5.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [31]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 4682350 rows, 270000 columns and 30195200 nonzeros
Model fingerprint: 0x17d52ad1
Variable types: 0 continuous, 270000 integer (270000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 16.0000000
Presolve removed 0 rows and 45300 columns (presolve time = 5s) ...
Presolve removed 4510210 rows and 262112 columns (presolve time = 10s) ...
Presolve removed 4533818 rows and 262192 columns (presolve time = 15s) ...
Presolve removed 4551855 rows and 262192 columns (presolve time = 20s) ...
Presolve removed 4560403 rows and 262192 columns (presolve time =

 19026 12004  109.60166   65  235  108.00000  109.76617  1.64%   254  440s
 19654 12138  109.43933   76  285  108.00000  109.76279  1.63%   254  447s
 19771 12317  109.15702   92  225  108.00000  109.76066  1.63%   255  450s
 20547 12595  109.51011   70  254  108.00000  109.75701  1.63%   255  457s
 20888 12790  109.25445   87  246  108.00000  109.75445  1.62%   256  460s
 21662 12962  109.09776   67  171  108.00000  109.74853  1.62%   255  468s
 22141 13088  109.27420   70  249  108.00000  109.74635  1.62%   255  472s
 22603 13262  109.18517   65  197  108.00000  109.74222  1.61%   255  476s
 23134 13663  109.23295  112  196  108.00000  109.73915  1.61%   254  480s
 24342 13934  109.43893   74  280  108.00000  109.73056  1.60%   253  489s
 24851 14215  109.12342   56  227  108.00000  109.72939  1.60%   253  493s
 25555 14262  109.37497   82  283  108.00000  109.72349  1.60%   252  503s
 26129 15010  109.50467   62  284  108.00000  109.71972  1.59%   251  508s
 27442 14875  109.00537  

## Entrada de datos: Instancia 4x150x3x1

In [32]:
datafile="4x150x3x1.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [33]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 4682350 rows, 270000 columns and 30195200 nonzeros
Model fingerprint: 0x7901a392
Variable types: 0 continuous, 270000 integer (270000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 12.0000000
Presolve removed 0 rows and 45300 columns (presolve time = 5s) ...
Presolve removed 4519707 rows and 262752 columns (presolve time = 12s) ...
Presolve removed 4546698 rows and 262848 columns (presolve time = 15s) ...
Presolve removed 4564123 rows and 262848 columns (presolve time = 20s) ...
Presolve removed 4573654 rows and 262848 columns (presolve time =

## Entrada de datos: Instancia 4x150x3x2

In [34]:
datafile="4x150x3x2.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [35]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 4682350 rows, 270000 columns and 30195200 nonzeros
Model fingerprint: 0x7021b7f1
Variable types: 0 continuous, 270000 integer (270000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 16.0000000
Presolve removed 0 rows and 45300 columns (presolve time = 5s) ...
Presolve removed 4541058 rows and 263476 columns (presolve time = 12s) ...
Presolve removed 4570653 rows and 263556 columns (presolve time = 15s) ...
Presolve removed 4586458 rows and 263556 columns (presolve time = 20s) ...
Presolve removed 4595225 rows and 263556 columns (presolve time =

     0     0   99.87625    0  379   97.00000   99.87625  2.97%     -   86s
     0     0   99.87556    0  349   97.00000   99.87556  2.96%     -   86s
     0     0   99.87514    0  385   97.00000   99.87514  2.96%     -   86s
     0     0   99.87503    0  375   97.00000   99.87503  2.96%     -   86s
     0     0   99.82709    0  374   97.00000   99.82709  2.91%     -   89s
     0     0   99.81903    0  388   97.00000   99.81903  2.91%     -   90s
     0     0   99.81802    0  363   97.00000   99.81802  2.91%     -   90s
     0     0   99.81777    0  374   97.00000   99.81777  2.90%     -   91s
     0     0   99.80361    0  372   97.00000   99.80361  2.89%     -   92s
     0     0   99.80229    0  334   97.00000   99.80229  2.89%     -   92s
     0     0   99.80195    0  347   97.00000   99.80195  2.89%     -   92s
     0     0   99.78333    0  350   97.00000   99.78333  2.87%     -   93s
     0     0   99.78333    0  372   97.00000   99.78333  2.87%     -   94s
     0     0   99.76778  

 12837  7553   99.25328   63  122   98.00000   99.25328  1.28%   407  490s
 13144  7742   99.25114   75  227   98.00000   99.25328  1.28%   404  495s
 13465  7915   99.07601   94  252   98.00000   99.25328  1.28%   401  500s
 14283  8274   99.00000  132  115   98.00000   99.25328  1.28%   389  505s
 14780  8569   99.25000   80  186   98.00000   99.25328  1.28%   384  510s
 15489  8827   99.20221  193  241   98.00000   99.25328  1.28%   378  516s
 15843  9033   99.15760  245  217   98.00000   99.25328  1.28%   376  520s
 16526  9167   99.25328   46  365   98.00000   99.25328  1.28%   374  526s
 16825  9455   99.13992   71  322   98.00000   99.25328  1.28%   375  530s
 17918  9851   99.01756  167  205   98.00000   99.25328  1.28%   364  537s
 18486 10059   99.03125   77  252   98.00000   99.25328  1.28%   363  541s
 19056 10477   99.00000  127   99   98.00000   99.25328  1.28%   361  546s
 19893 10987   99.00000   52  104   98.00000   99.25328  1.28%   357  551s
 20815 11251   99.00000  

## Entrada de datos: Instancia 4x150x3x3

In [36]:
datafile="4x150x3x3.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [37]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 4682350 rows, 270000 columns and 30195200 nonzeros
Model fingerprint: 0x1b1af4de
Variable types: 0 continuous, 270000 integer (270000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 17.0000000
Presolve removed 0 rows and 45300 columns (presolve time = 5s) ...
Presolve removed 4537398 rows and 263060 columns (presolve time = 12s) ...
Presolve removed 4565110 rows and 263128 columns (presolve time = 15s) ...
Presolve removed 4578188 rows and 263128 columns (presolve time = 20s) ...
Presolve removed 4585515 rows and 263128 columns (presolve time =

     0     0  104.15871    0  591  101.00000  104.15871  3.13%     -  114s
     0     0  104.15748    0  606  101.00000  104.15748  3.13%     -  114s
     0     0  104.15722    0  598  101.00000  104.15722  3.13%     -  115s
     0     0  104.05349    0  580  101.00000  104.05349  3.02%     -  117s
     0     0  104.05258    0  604  101.00000  104.05258  3.02%     -  117s
     0     0  104.05042    0  590  101.00000  104.05042  3.02%     -  118s
     0     0  104.05024    0  600  101.00000  104.05024  3.02%     -  118s
     0     0  104.04370    0  571  101.00000  104.04370  3.01%     -  119s
     0     0  104.04240    0  610  101.00000  104.04240  3.01%     -  120s
     0     0  104.04228    0  607  101.00000  104.04228  3.01%     -  120s
     0     0  104.03529    0  611  101.00000  104.03529  3.01%     -  121s
     0     0  104.03217    0  624  101.00000  104.03217  3.00%     -  122s
     0     0  104.03149    0  622  101.00000  104.03149  3.00%     -  122s
     0     0  104.01414  

  6908  6364  103.14858   93  330  102.00000  103.44914  1.42%   369  620s
  7026  6434  103.32695  102  337  102.00000  103.44914  1.42%   381  640s
  7074  6480  103.17478  105  331  102.00000  103.44914  1.42%   383  646s
  7140  6560  103.17093  111  306  102.00000  103.44914  1.42%   386  651s
  7253  6651  103.16941  122  316  102.00000  103.44914  1.42%   392  656s
  7386  6684  103.15127  128  327  102.00000  103.44914  1.42%   396  660s
  7478  6702  103.11876  137  265  102.00000  103.44914  1.42%   402  665s
  7555  6783  103.11617  149  294  102.00000  103.44914  1.42%   412  670s
  7810  6928  103.09381  167  275  102.00000  103.44914  1.42%   422  679s
  7951  6990  103.08476  174  280  102.00000  103.44907  1.42%   427  685s
  8080  7128  103.38776   74  431  102.00000  103.44907  1.42%   432  690s
  8265  7242  103.08828   91  371  102.00000  103.44907  1.42%   436  696s
  8453  7334  103.04632  115  358  102.00000  103.44907  1.42%   440  701s
  8628  7447  103.02649  

 14974 10713  103.22891  158  497  102.00000  103.34331  1.32%   611 1459s
 15135 10776  103.25899  164  511  102.00000  103.34331  1.32%   610 1467s
 15291 10799  103.24481  169  549  102.00000  103.34331  1.32%   611 1475s
 15375 10867  103.19580  172  496  102.00000  103.34331  1.32%   612 1483s
 15515 10960     cutoff  177       102.00000  103.34331  1.32%   613 1491s
 15692 11064  103.15878  182  522  102.00000  103.34331  1.32%   613 1499s
 15918 11102  103.15130  187  498  102.00000  103.34331  1.32%   610 1507s
 16041 11160  103.11907  191  498  102.00000  103.34331  1.32%   610 1517s
 16159 11347  103.10406  197  486  102.00000  103.34331  1.32%   610 1527s
 16441 11407  103.09406  208  470  102.00000  103.34331  1.32%   606 1537s
 16655 11500  103.08516  221  430  102.00000  103.34331  1.32%   604 1548s
 16880 11561  103.06833  240  426  102.00000  103.33790  1.31%   602 1558s
 17081 11623  103.28032   73  580  102.00000  103.33790  1.31%   602 1572s
 17231 11724  103.22132  

 38854 22307  103.21467  104  710  102.00000  103.30130  1.28%   619 3345s
 38857 22309  103.16083  172  699  102.00000  103.30074  1.28%   619 3352s
 38859 22311  103.26371   78  707  102.00000  103.30035  1.27%   619 3355s
 38861 22312  103.18254  111  708  102.00000  103.29993  1.27%   619 3360s
 38864 22314  103.10933  117  702  102.00000  103.29954  1.27%   619 3365s
 38866 22315  103.01363  180  680  102.00000  103.29914  1.27%   619 3370s
 38868 22317  103.29745  106  703  102.00000  103.29895  1.27%   619 3375s
 38871 22319  103.04240  200  668  102.00000  103.29834  1.27%   619 3381s
 38874 22321  103.08144  204  707  102.00000  103.29807  1.27%   619 3386s
 38877 22323  103.18945   80  722  102.00000  103.29774  1.27%   619 3391s
 38880 22325  103.02407  282  712  102.00000  103.29757  1.27%   619 3396s
 38883 22327  103.10282   95  725  102.00000  103.29728  1.27%   619 3401s
 38885 22328  103.00304  235  716  102.00000  103.29706  1.27%   619 3406s
 38888 22330  103.10069  

## Entrada de datos: Instancia 4x150x3x4

In [38]:
datafile="4x150x3x4.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [39]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 4682350 rows, 270000 columns and 30195200 nonzeros
Model fingerprint: 0xeb7894ea
Variable types: 0 continuous, 270000 integer (270000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 14.0000000
Presolve removed 0 rows and 45300 columns (presolve time = 5s) ...
Presolve removed 4542598 rows and 263408 columns (presolve time = 12s) ...
Presolve removed 4570479 rows and 263480 columns (presolve time = 15s) ...
Presolve removed 4587082 rows and 263480 columns (presolve time = 20s) ...
Presolve removed 4595718 rows and 263480 columns (presolve time =

## Entrada de datos: Instancia 4x150x3x5

In [40]:
datafile="4x150x3x5.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [41]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 4682350 rows, 270000 columns and 30195200 nonzeros
Model fingerprint: 0x36cafd84
Variable types: 0 continuous, 270000 integer (270000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 12.0000000
Presolve removed 0 rows and 45300 columns (presolve time = 5s) ...
Presolve removed 4537583 rows and 263400 columns (presolve time = 12s) ...
Presolve removed 4565384 rows and 263472 columns (presolve time = 15s) ...
Presolve removed 4584225 rows and 263472 columns (presolve time = 20s) ...
Presolve removed 4592596 rows and 263472 columns (presolve time =

 16291  2981   96.00000   49  130   95.00000   96.00000  1.05%   256  362s
 17769  4358 infeasible   58        95.00000   96.00000  1.05%   258  376s
 21376  4294 infeasible   89        95.00000   96.00000  1.05%   236  385s
 22693  4018   96.00000   66   82   95.00000   96.00000  1.05%   231  392s
 23932  3876 infeasible   55        95.00000   96.00000  1.05%   227  400s
 24746  4046   96.00000   53   84   95.00000   96.00000  1.05%   228  408s
 25255  4047   96.00000   80   73   95.00000   96.00000  1.05%   231  415s
 25268  4056   96.00000   74   12   95.00000   96.00000  1.05%   230  420s
 25287  4072   96.00000   27  134   95.00000   96.00000  1.05%   232  425s
 25400  4116   96.00000   42   75   95.00000   96.00000  1.05%   231  433s
 25452  4163   96.00000   48   69   95.00000   96.00000  1.05%   231  438s
 25581  4352   96.00000   67   95   95.00000   96.00000  1.05%   230  440s
 25975  4678   96.00000   97   67   95.00000   96.00000  1.05%   228  445s
 27405  4400   96.00000  

## Entrada de datos: Instancia 4x150x3.5x1

In [42]:
datafile="4x150x3.5x1.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [43]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Academic license - for non-commercial use only - expires 2021-06-11
Using license file C:\Users\elPatan03\gurobi.lic
Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 4682350 rows, 270000 columns and 30195200 nonzeros
Model fingerprint: 0xe2718c57
Variable types: 0 continuous, 270000 integer (270000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 13.0000000
Presolve removed 0 rows and 45300 columns (presolve time = 5s) ...
Presolve removed 4548800 rows and 263836 columns (presolve time = 12s) ...
Presolve removed 4575912 rows and 263912 columns (presolve time = 15s) ...
Presolve removed 458756

## Entrada de datos: Instancia 4x150x3.5x2

In [44]:
datafile="4x150x3.5x2.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [45]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 4682350 rows, 270000 columns and 30195200 nonzeros
Model fingerprint: 0xc3ae53a5
Variable types: 0 continuous, 270000 integer (270000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 17.0000000
Presolve removed 0 rows and 45300 columns (presolve time = 5s) ...
Presolve removed 4551332 rows and 264020 columns (presolve time = 12s) ...
Presolve removed 4579918 rows and 264084 columns (presolve time = 15s) ...
Presolve removed 4595574 rows and 264084 columns (presolve time = 20s) ...
Presolve removed 4603813 rows and 264084 columns (presolve time =

## Entrada de datos: Instancia 4x150x3.5x3

In [46]:
datafile="4x150x3.5x3.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [47]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 4682350 rows, 270000 columns and 30195200 nonzeros
Model fingerprint: 0x184ca595
Variable types: 0 continuous, 270000 integer (270000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 14.0000000
Presolve removed 0 rows and 45300 columns (presolve time = 5s) ...
Presolve removed 4555012 rows and 264180 columns (presolve time = 12s) ...
Presolve removed 4583960 rows and 264268 columns (presolve time = 15s) ...
Presolve removed 4597246 rows and 264268 columns (presolve time = 20s) ...
Presolve removed 4604690 rows and 264268 columns (presolve time =

     0     0   85.50000    0  229   84.00000   85.50000  1.79%     -   88s
     0     0   85.50000    0  123   84.00000   85.50000  1.79%     -   89s
     0     0   85.50000    0  221   84.00000   85.50000  1.79%     -   89s
     0     0   85.50000    0  143   84.00000   85.50000  1.79%     -   90s
     0     0   85.50000    0  132   84.00000   85.50000  1.79%     -   90s
     0     2   85.50000    0   85   84.00000   85.50000  1.79%     -   91s
   183   186   85.50000   18  195   84.00000   85.50000  1.79%   199   95s
  1239  1045   85.10348   36  234   84.00000   85.50000  1.79%   141  100s
  1344  1050   85.00000   26  326   84.00000   85.00000  1.19%   139  105s
  1357  1059   85.00000    9   42   84.00000   85.00000  1.19%   138  110s
  1372  1070   85.00000   42  192   84.00000   85.00000  1.19%   168  115s
  1387  1080   85.00000   76   71   84.00000   85.00000  1.19%   167  120s
  1487  1280   85.00000   35   48   84.00000   85.00000  1.19%   182  129s
  1619  1791   85.00000  

## Entrada de datos: Instancia 4x150x3.5x4

In [48]:
datafile="4x150x3.5x4.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [49]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 4682350 rows, 270000 columns and 30195200 nonzeros
Model fingerprint: 0xfcea3583
Variable types: 0 continuous, 270000 integer (270000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 12.0000000
Presolve removed 0 rows and 45300 columns (presolve time = 5s) ...
Presolve removed 4536975 rows and 263712 columns (presolve time = 12s) ...
Presolve removed 4566014 rows and 263792 columns (presolve time = 15s) ...
Presolve removed 4581583 rows and 263792 columns (presolve time = 20s) ...
Presolve removed 4589578 rows and 263792 columns (presolve time =

     0     0   87.01179    0  285   85.00000   87.01179  2.37%     -  103s
     0     0   87.01072    0  260   85.00000   87.01072  2.37%     -  104s
     0     0   87.00975    0  242   85.00000   87.00975  2.36%     -  104s
     0     0   87.00975    0  266   85.00000   87.00975  2.36%     -  104s
     0     0   87.00000    0  231   85.00000   87.00000  2.35%     -  104s
     0     0   86.99993    0  295   85.00000   86.99993  2.35%     -  105s
     0     0   86.99654    0  290   85.00000   86.99654  2.35%     -  105s
     0     0   86.99649    0  296   85.00000   86.99649  2.35%     -  105s
     0     0   86.98981    0  312   85.00000   86.98981  2.34%     -  106s
     0     0   86.98927    0  278   85.00000   86.98927  2.34%     -  106s
     0     0   86.98874    0  280   85.00000   86.98874  2.34%     -  107s
     0     0   86.98868    0  254   85.00000   86.98868  2.34%     -  107s
     0     0   86.97414    0  278   85.00000   86.97414  2.32%     -  107s
     0     0   86.97246  

## Entrada de datos: Instancia 4x150x3.5x5

In [50]:
datafile="4x150x3.5x5.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [51]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 4682350 rows, 270000 columns and 30195200 nonzeros
Model fingerprint: 0x60d99bf4
Variable types: 0 continuous, 270000 integer (270000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 14.0000000
Presolve removed 0 rows and 45300 columns (presolve time = 5s) ...
Presolve removed 4551664 rows and 264100 columns (presolve time = 10s) ...
Presolve removed 4577080 rows and 264180 columns (presolve time = 15s) ...
Presolve removed 4594362 rows and 264180 columns (presolve time = 20s) ...
Presolve removed 4604139 rows and 264180 columns (presolve time =

## Entrada de datos: Instancia 4x150x4x1

In [52]:
datafile="4x150x4x1.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [53]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 4682350 rows, 270000 columns and 30195200 nonzeros
Model fingerprint: 0xeadfb402
Variable types: 0 continuous, 270000 integer (270000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 17.0000000
Presolve removed 0 rows and 45300 columns (presolve time = 5s) ...
Presolve removed 4559510 rows and 264440 columns (presolve time = 12s) ...
Presolve removed 4585750 rows and 264500 columns (presolve time = 15s) ...
Presolve removed 4602528 rows and 264500 columns (presolve time = 20s) ...
Presolve removed 4610556 rows and 264500 columns (presolve time =

     0     0   79.92875    0  268   78.00000   79.92875  2.47%     -   78s
     0     0   79.92804    0  258   78.00000   79.92804  2.47%     -   78s
     0     0   79.92787    0  281   78.00000   79.92787  2.47%     -   78s
     0     0   79.92023    0  290   78.00000   79.92023  2.46%     -   78s
     0     0   79.91409    0  278   78.00000   79.91409  2.45%     -   79s
     0     0   79.91346    0  284   78.00000   79.91346  2.45%     -   79s
     0     0   79.91346    0  282   78.00000   79.91346  2.45%     -   79s
     0     0   79.89714    0  244   78.00000   79.89714  2.43%     -   79s
     0     0   79.89651    0  269   78.00000   79.89651  2.43%     -   79s
     0     0   79.89627    0  251   78.00000   79.89627  2.43%     -   79s
     0     0   79.89138    0  262   78.00000   79.89138  2.42%     -   80s
     0     0   79.89005    0  287   78.00000   79.89005  2.42%     -   80s
     0     0   79.88951    0  277   78.00000   79.88951  2.42%     -   80s
     0     0   79.88902  

## Entrada de datos: Instancia 4x150x4x2

In [54]:
datafile="4x150x4x2.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [55]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 4682350 rows, 270000 columns and 30195200 nonzeros
Model fingerprint: 0xd2c3c41b
Variable types: 0 continuous, 270000 integer (270000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 12.0000000
Presolve removed 0 rows and 45300 columns (presolve time = 5s) ...
Presolve removed 4567185 rows and 264952 columns (presolve time = 12s) ...
Presolve removed 4598118 rows and 265008 columns (presolve time = 15s) ...
Presolve removed 4612385 rows and 265008 columns (presolve time = 20s) ...
Presolve removed 4620322 rows and 265008 columns (presolve time =

## Entrada de datos: Instancia 4x150x4x3

In [56]:
datafile="4x150x4x3.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [57]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 4682350 rows, 270000 columns and 30195200 nonzeros
Model fingerprint: 0x1f1bdacb
Variable types: 0 continuous, 270000 integer (270000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 13.0000000
Presolve removed 0 rows and 45300 columns (presolve time = 5s) ...
Presolve removed 4546550 rows and 264184 columns (presolve time = 12s) ...
Presolve removed 4574851 rows and 264252 columns (presolve time = 15s) ...
Presolve removed 4594411 rows and 264252 columns (presolve time = 20s) ...
Presolve removed 4602690 rows and 264252 columns (presolve time =

## Entrada de datos: Instancia 4x150x4x4

In [58]:
datafile="4x150x4x4.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [59]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 4682350 rows, 270000 columns and 30195200 nonzeros
Model fingerprint: 0xf7224324
Variable types: 0 continuous, 270000 integer (270000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 14.0000000
Presolve removed 0 rows and 45300 columns (presolve time = 5s) ...
Presolve removed 4572787 rows and 264876 columns (presolve time = 12s) ...
Presolve removed 4601442 rows and 264932 columns (presolve time = 15s) ...
Presolve removed 4614588 rows and 264932 columns (presolve time = 20s) ...
Presolve removed 4621246 rows and 264932 columns (presolve time =

## Entrada de datos: Instancia 4x150x4x5

In [60]:
datafile="4x150x4x5.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [61]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 4682350 rows, 270000 columns and 30195200 nonzeros
Model fingerprint: 0x994c1df1
Variable types: 0 continuous, 270000 integer (270000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 14.0000000
Presolve removed 0 rows and 45300 columns (presolve time = 6s) ...
Presolve removed 4558563 rows and 264540 columns (presolve time = 10s) ...
Presolve removed 4580860 rows and 264604 columns (presolve time = 15s) ...
Presolve removed 4599866 rows and 264604 columns (presolve time = 20s) ...
Presolve removed 4608689 rows and 264604 columns (presolve time =

## Entrada de datos: Instancia 4x150x4.5x1

In [62]:
datafile="4x150x4.5x1.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [63]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 4682350 rows, 270000 columns and 30195200 nonzeros
Model fingerprint: 0x8f1eff55
Variable types: 0 continuous, 270000 integer (270000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 13.0000000
Presolve removed 0 rows and 45300 columns (presolve time = 5s) ...
Presolve removed 4560534 rows and 264844 columns (presolve time = 12s) ...
Presolve removed 4586867 rows and 264908 columns (presolve time = 15s) ...
Presolve removed 4602223 rows and 264908 columns (presolve time = 20s) ...
Presolve removed 4611624 rows and 264908 columns (presolve time =

## Entrada de datos: Instancia 4x150x4.5x2

In [64]:
datafile="4x150x4.5x2.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [65]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 4682350 rows, 270000 columns and 30195200 nonzeros
Model fingerprint: 0xe42d4908
Variable types: 0 continuous, 270000 integer (270000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 14.0000000
Presolve removed 0 rows and 45300 columns (presolve time = 5s) ...
Presolve removed 4578356 rows and 265200 columns (presolve time = 12s) ...
Presolve removed 4606345 rows and 265244 columns (presolve time = 15s) ...
Presolve removed 4618132 rows and 265244 columns (presolve time = 20s) ...
Presolve removed 4625002 rows and 265244 columns (presolve time =

 38180 20815   76.00000   99   76   75.00000   76.00000  1.33%   176  400s
 39045 21352   76.00000  159  145   75.00000   76.00000  1.33%   176  408s
 39785 21907   76.00000   83  103   75.00000   76.00000  1.33%   175  412s
 40772 22153   76.00000   82  146   75.00000   76.00000  1.33%   174  415s
 42985 22360     cutoff  103        75.00000   76.00000  1.33%   171  421s
 43480 22286     cutoff   97        75.00000   76.00000  1.33%   174  427s
 43690 22343   76.00000   98  212   75.00000   76.00000  1.33%   175  430s
 44155 22365   76.00000   94  197   75.00000   76.00000  1.33%   178  436s
 44644 22339   76.00000   85  187   75.00000   76.00000  1.33%   182  443s
 44884 22322   76.00000   96  186   75.00000   76.00000  1.33%   183  446s
 45115 22341   76.00000   88  208   75.00000   76.00000  1.33%   185  450s
 45658 22576     cutoff   96        75.00000   76.00000  1.33%   189  458s
 46021 22602     cutoff  113        75.00000   76.00000  1.33%   190  462s
 46363 22707   76.00000  

 134303 40596     cutoff  144        75.00000   76.00000  1.33%   290 1392s
 134973 40758     cutoff  164        75.00000   76.00000  1.33%   290 1402s
 135471 41023   76.00000  145  173   75.00000   76.00000  1.33%   291 1412s
 136274 41254     cutoff  159        75.00000   76.00000  1.33%   291 1422s
 137199 41566   76.00000  124  221   75.00000   76.00000  1.33%   291 1432s
 138145 41826   76.00000  165  261   75.00000   76.00000  1.33%   292 1442s
 139051 42061   76.00000  164  150   75.00000   76.00000  1.33%   292 1452s
 139890 42317     cutoff  162        75.00000   76.00000  1.33%   292 1462s
 140732 42559     cutoff  150        75.00000   76.00000  1.33%   293 1472s
 141684 42882   76.00000  188  268   75.00000   76.00000  1.33%   293 1482s
 142563 43100     cutoff  151        75.00000   76.00000  1.33%   294 1492s
 143483 43228   76.00000  136  167   75.00000   76.00000  1.33%   294 1502s
 144231 43443     cutoff  159        75.00000   76.00000  1.33%   294 1512s
 145156 4360

 229067 73634   76.00000  133  166   75.00000   76.00000  1.33%   317 2563s
 229983 74139   76.00000  149  148   75.00000   76.00000  1.33%   317 2575s
 231018 74519   76.00000  153  155   75.00000   76.00000  1.33%   317 2587s
 232066 75043   76.00000  154  150   75.00000   76.00000  1.33%   318 2600s
 233003 75538   76.00000  160  216   75.00000   76.00000  1.33%   318 2611s
 233798 76040   76.00000  161   85   75.00000   76.00000  1.33%   318 2622s
 234672 76409     cutoff  182        75.00000   76.00000  1.33%   318 2634s
 235678 76782     cutoff  176        75.00000   76.00000  1.33%   318 2645s
 236832 77127     cutoff  169        75.00000   76.00000  1.33%   318 2656s
 237825 77488     cutoff  160        75.00000   76.00000  1.33%   318 2666s
 238922 77808     cutoff  172        75.00000   76.00000  1.33%   318 2676s
 239916 78078     cutoff  118        75.00000   76.00000  1.33%   318 2686s
 240864 78426     cutoff  158        75.00000   76.00000  1.33%   318 2696s
 241869 7874

## Entrada de datos: Instancia 4x150x4.5x3

In [66]:
datafile="4x150x4.5x3.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [67]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 4682350 rows, 270000 columns and 30195200 nonzeros
Model fingerprint: 0xa45b09c5
Variable types: 0 continuous, 270000 integer (270000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 14.0000000
Presolve removed 0 rows and 45300 columns (presolve time = 5s) ...
Presolve removed 4571866 rows and 265016 columns (presolve time = 12s) ...
Presolve removed 4598833 rows and 265076 columns (presolve time = 15s) ...
Presolve removed 4609204 rows and 265076 columns (presolve time = 20s) ...
Presolve removed 4614754 rows and 265076 columns (presolve time =

 14473   707     cutoff   46        67.00000   68.00000  1.49%   466  353s
 15255   561     cutoff   31        67.00000   68.00000  1.49%   472  367s
 15657   413     cutoff   43        67.00000   68.00000  1.49%   483  392s
 16901   297   68.00000   41  214   67.00000   68.00000  1.49%   485  428s
 17373    58   68.00000   42  200   67.00000   68.00000  1.49%   487  446s
 18573    15     cutoff   34        67.00000   68.00000  1.49%   492  457s
 18984     0     cutoff   31        67.00000   68.00000  1.49%   493  460s

Cutting planes:
  Gomory: 26
  Cover: 269
  Projected implied bound: 1
  Clique: 12
  MIR: 331
  StrongCG: 212
  Flow cover: 47
  GUB cover: 11
  Inf proof: 2
  Zero half: 30
  Relax-and-lift: 1

Explored 19114 nodes (9490940 simplex iterations) in 461.22 seconds
Thread count was 6 (of 6 available processors)

Solution count 7: 67 66 64 ... -0

Optimal solution found (tolerance 1.00e-04)
Best objective 6.700000000000e+01, best bound 6.700000000000e+01, gap 0.0000%


## Entrada de datos: Instancia 4x150x4.5x4

In [68]:
datafile="4x150x4.5x4.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [69]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 4682350 rows, 270000 columns and 30195200 nonzeros
Model fingerprint: 0xdf8dfbdb
Variable types: 0 continuous, 270000 integer (270000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 13.0000000
Presolve removed 0 rows and 45300 columns (presolve time = 5s) ...
Presolve removed 4568172 rows and 264888 columns (presolve time = 12s) ...
Presolve removed 4591441 rows and 264948 columns (presolve time = 15s) ...
Presolve removed 4607489 rows and 264948 columns (presolve time = 20s) ...
Presolve removed 4615727 rows and 264948 columns (presolve time =

## Entrada de datos: Instancia 4x150x4.5x5

In [70]:
datafile="4x150x4.5x5.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [71]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 4682350 rows, 270000 columns and 30195200 nonzeros
Model fingerprint: 0xd5742841
Variable types: 0 continuous, 270000 integer (270000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 12.0000000
Presolve removed 0 rows and 45300 columns (presolve time = 5s) ...
Presolve removed 4561352 rows and 264888 columns (presolve time = 12s) ...
Presolve removed 4589114 rows and 264932 columns (presolve time = 15s) ...
Presolve removed 4604822 rows and 264932 columns (presolve time = 20s) ...
Presolve removed 4613523 rows and 264932 columns (presolve time =

## Entrada de datos: Instancia 4x150x5x1

In [72]:
datafile="4x150x5x1.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [73]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 4682350 rows, 270000 columns and 30195200 nonzeros
Model fingerprint: 0xbe27befc
Variable types: 0 continuous, 270000 integer (270000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 13.0000000
Presolve removed 0 rows and 45300 columns (presolve time = 5s) ...
Presolve removed 4570388 rows and 265112 columns (presolve time = 12s) ...
Presolve removed 4596917 rows and 265156 columns (presolve time = 15s) ...
Presolve removed 4612722 rows and 265156 columns (presolve time = 20s) ...
Presolve removed 4620951 rows and 265156 columns (presolve time =

 29291  8299   69.00000   59  219   68.00000   69.00000  1.47%   476  690s
 29615  8337   69.00000   61  102   68.00000   69.00000  1.47%   477  696s
 29921  8375   69.00000   68  157   68.00000   69.00000  1.47%   480  703s
 30056  8416 infeasible   80        68.00000   69.00000  1.47%   482  706s
 30222  8559   69.00000   74  154   68.00000   69.00000  1.47%   484  710s
 30764  8629 infeasible   67        68.00000   69.00000  1.47%   484  718s
 30986  8768   69.00000   83  148   68.00000   69.00000  1.47%   487  722s
 31341  8908   69.00000   50  212   68.00000   69.00000  1.47%   485  727s
 31865  8984   69.00000   56   90   68.00000   69.00000  1.47%   485  733s
 32486  8927   69.00000   70  164   68.00000   69.00000  1.47%   483  738s
 32910  8899   69.00000   72  131   68.00000   69.00000  1.47%   484  743s
 33193  8955   69.00000   71   89   68.00000   69.00000  1.47%   486  749s
 33684  8910 infeasible   73        68.00000   69.00000  1.47%   485  755s
 34044  9118   69.00000  

## Entrada de datos: Instancia 4x150x5x2

In [74]:
datafile="4x150x5x2.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [75]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]
WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 4682350 rows, 270000 columns and 30195200 nonzeros
Model fingerprint: 0xa5014ef2
Variable types: 0 continuous, 270000 integer (270000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 13.0000000
Presolve removed 0 rows and 45300 columns (presolve time = 5s) ...
Presolve removed 4573758 rows and 265260 columns (presolve time = 12s) ...
Presolve removed 4603398 rows and 265308 columns (presolve time = 15s) ...
Presolve removed 4612397 rows and 265308 columns (presolve time = 20s) ...
Presolve removed 4618733 rows and 265308 columns (presolve time =

## Entrada de datos: Instancia 4x150x5x3

In [76]:
datafile="4x150x5x3.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [77]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]
WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 4682350 rows, 270000 columns and 30195200 nonzeros
Model fingerprint: 0x4daaeddd
Variable types: 0 continuous, 270000 integer (270000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 14.0000000
Presolve removed 0 rows and 45300 columns (presolve time = 5s) ...
Presolve removed 4568664 rows and 265128 columns (presolve time = 12s) ...
Presolve removed 4597888 rows and 265176 columns (presolve time = 15s) ...
Presolve removed 4613970 rows and 265176 columns (presolve time = 20s) ...
Presolve removed 4622992 rows and 265176 columns (presolve time =

## Entrada de datos: Instancia 4x150x5x4

In [78]:
datafile="4x150x5x4.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [79]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]
WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 4682350 rows, 270000 columns and 30195200 nonzeros
Model fingerprint: 0x35323d96
Variable types: 0 continuous, 270000 integer (270000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 13.0000000
Presolve removed 0 rows and 45300 columns (presolve time = 5s) ...
Presolve removed 4572321 rows and 265152 columns (presolve time = 12s) ...
Presolve removed 4598279 rows and 265200 columns (presolve time = 15s) ...
Presolve removed 4612581 rows and 265200 columns (presolve time = 20s) ...
Presolve removed 4621827 rows and 265200 columns (presolve time =

## Entrada de datos: Instancia 4x150x5x5

In [80]:
datafile="4x150x5x5.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

150

In [81]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 4682350 rows, 270000 columns and 30195200 nonzeros
Model fingerprint: 0x99f0be3c
Variable types: 0 continuous, 270000 integer (270000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 16.0000000
Presolve removed 0 rows and 45300 columns (presolve time = 5s) ...
Presolve removed 4584533 rows and 265528 columns (presolve time = 12s) ...
Presolve removed 4610240 rows and 265576 columns (presolve time = 15s) ...
Presolve removed 4630494 rows and 265576 columns (presolve time = 20s) ...
Presolve removed 4639766 rows and 265576 columns (presolve time =